In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score, recall_score
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import torch_geometric.transforms as T
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

In [ ]:

df = pd.read_csv("final_clean.csv")

df = df[['InvoiceNo', 'stockCodeTransform', 'CustomerID', 'Quantity']]

df = df[df['Quantity'] > 0]

customer_product_matrix = df.pivot_table(
    index='CustomerID',
    columns='stockCodeTransform',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

customer_product_matrix = (customer_product_matrix > 0).astype(int)

train_ids, test_ids = train_test_split(customer_product_matrix.index, test_size=0.2, random_state=42)

train_matrix = customer_product_matrix.loc[train_ids]
test_matrix = customer_product_matrix.loc[test_ids]


In [25]:
data_matrix = customer_product_matrix

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch_geometric.data import Data
from tqdm import tqdm

def create_graph_from_data(df):
    # Tiền xử lý: Mã hóa các cột cần thiết
    le_country = LabelEncoder()
    le_stock = LabelEncoder()
    df["Country"] = le_country.fit_transform(df["Country"])
    df["StockCode"] = le_stock.fit_transform(df["StockCode"])

    # Chuẩn hóa các cột liên quan đến sản phẩm
    scaler = MinMaxScaler()
    df[["UnitPrice", "Utility", "rate"]] = scaler.fit_transform(df[["UnitPrice", "Utility", "rate"]])

    # Tạo các đặc trưng cho khách hàng (Customer Features)
    customer_features = df.groupby("CustomerID").agg(
        {"Country": "mean", "rate": "mean"}
    ).reset_index()
    customer_features_tensor = torch.tensor(customer_features.iloc[:, 1:].values, dtype=torch.float)

    # Tạo các đặc trưng cho sản phẩm (Product Features)
    product_features = df.groupby("StockCode").agg(
        {"UnitPrice": "mean", "Utility": "mean"}
    ).reset_index()
    product_features_tensor = torch.tensor(product_features.iloc[:, 1:].values, dtype=torch.float)

    # Tạo danh sách tất cả các kết hợp (CustomerID, StockCode)
    customers = df['CustomerID'].unique()
    products = df['stockCodeTransform'].unique()

    # Tạo một dictionary để xác định xem khách hàng có mua sản phẩm này không
    purchase_dict = {(row['CustomerID'], row['stockCodeTransform']): 1 for _, row in df.iterrows()}

    # Tạo đặc trưng cho các kết hợp (CustomerID, StockCode)
    x1 = []
    y1 = []

    # Lặp qua tất cả các khách hàng và sản phẩm để xây dựng đặc trưng cho mỗi cặp
    for customer in tqdm(customers, desc="Processing customers"):
        for product in products:
            # Lấy đặc trưng của khách hàng và sản phẩm
            customer_feature = customer_features_tensor[customer_features["CustomerID"] == customer][0]
            product_feature = product_features_tensor[product_features["StockCode"] == product][0]

            # Ghép đặc trưng của khách hàng và sản phẩm thành một vector
            combined_feature = list(customer_feature) + list(product_feature)
            x1.append(combined_feature)

            # Gán nhãn: Nếu cặp (customer, product) có trong purchase_dict thì gán nhãn = 1, ngược lại = 0
            label = purchase_dict.get((customer, product), 0)
            y1.append(label)

    # Chuyển x và y thành tensor của PyTorch
    x1_tensor = torch.tensor(np.array(x1), dtype=torch.float)
    y_tensor = torch.tensor(np.array(y1), dtype=torch.long)

    # Dữ liệu mô phỏng về danh sách khách hàng và sản phẩm để tạo nhãn
    customers = df['CustomerID'].unique()
    products = df['stockCodeTransform'].unique()

    # Tạo danh sách tất cả các kết hợp (CustomerID, StockCode)
    customer_product_combinations = [(customer, product) for customer in customers for product in products]
    # Tạo danh sách các cạnh và trọng số
    edges = []
    edge_attr = []  # Trọng số cho các cạnh
    customer_nodes = len(customers)  # Số lượng customer nodes
    # Duyệt qua các kết hợp (CustomerID, StockCode)
    for i, combination in enumerate(customer_product_combinations):
        customer_id, stock_code = combination
        # Nếu khách hàng đã mua sản phẩm này, trọng số = 1, nếu không, trọng số = 0
        weight = 1 if (customer_id, stock_code) in purchase_dict else 0
        edges.append((customer_id, stock_code))
        edge_attr.append(weight)

    # Chuyển edges thành tensor và edge_attr thành tensor
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Tạo Data cho đồ thị (graph)
    graph = Data(x=x1_tensor, edge_index=edge_index, edge_attr=edge_attr, y=y_tensor)

    return graph

# Ví dụ sử dụng hàm
df = pd.read_csv("final_clean.csv")
graph = create_graph_from_data(df)

# Kiểm tra cấu trúc dữ liệu
print("Graph:")
print(graph)


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
graph_train = create_graph_from_data(df_train)
graph_test = create_graph_from_data(df_test)

In [ ]:
print(graph_train)

Data(x=[12563443, 4], edge_index=[2, 12564131], edge_attr=[12564131], y=[12563443])


In [ ]:
print(graph_test)

Data(x=[3140861, 4], edge_index=[2, 3141032], edge_attr=[3141032], y=[3140861])


In [9]:
class GATModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GATModel, self).__init__()
        # Giảm số lượng đầu vào (heads) và lớp ẩn
        self.gat1 = GATConv(num_features, hidden_channels, heads=8, dropout=0.6)
        self.gat2 = GATConv(hidden_channels * 8, num_classes, dropout=0.6)

    def forward(self, x, edge_index, edge_attr=None):
        x = F.elu(self.gat1(x, edge_index, edge_attr))
        x = self.gat2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)


In [ ]:
num_features = graph_train.x.size(1)
hidden_channels = 64
num_classes = 2
model = GATModel(num_features, hidden_channels, num_classes)

class_weights = torch.tensor([1.0, 10])
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = Adam(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

In [ ]:
import torch

# Huấn luyện mô hình với Early Stopping
epochs = 300
patience = 30  # Số epoch không cải thiện loss trước khi dừng lại
best_loss = float('inf')  # Khởi tạo loss tốt nhất là vô cùng lớn
best_accuracy = 0  # Khởi tạo accuracy tốt nhất là 0
epochs_without_improvement = 0  # Đếm số epoch không cải thiện loss

for epoch in range(epochs):
    model.train()  # Đặt mô hình ở chế độ train
    optimizer.zero_grad()  # Làm mới gradient
    
    # Tính toán đầu ra của mô hình
    out = model(graph_train.x, graph_train.edge_index)  
    
    # Tính toán loss
    loss = criterion(out, graph_train.y)  
    
    # Lan truyền gradient và cập nhật tham số của mô hình
    loss.backward()  
    optimizer.step()  
    
    # Tính toán accuracy
    _, predicted = torch.max(out, 1)  # Lấy chỉ số của lớp dự đoán có xác suất cao nhất
    correct = (predicted == graph_train.y).sum().item()  # Tính số lượng dự đoán đúng
    accuracy = correct / graph_train.y.size(0)  # Tính accuracy
    
    # Kiểm tra xem loss có cải thiện hay không
    if loss.item() < best_loss:
        best_loss = loss.item()  # Cập nhật loss tốt nhất
        best_accuracy = accuracy  # Cập nhật accuracy tốt nhất
        epochs_without_improvement = 0  # Reset số epoch không cải thiện
        # Lưu mô hình tốt nhất và đè lên tệp hiện tại
        torch.save(model.state_dict(), "best_model_no_scale_1_64.pth")
        print(f"Saved best model at epoch {epoch+1} with loss: {loss:.4f}, accuracy: {accuracy * 100:.2f}%")
    else:
        epochs_without_improvement += 1  # Tăng số epoch không cải thiện
    
    # In ra loss và accuracy mỗi 10 epoch
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")
    
    # Kiểm tra điều kiện Early Stopping
    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch+1}/{epochs} due to no improvement.")
        break

# In ra thông tin về mô hình tốt nhất
print(f"Best model has loss: {best_loss:.4f} and accuracy: {best_accuracy * 100:.2f}%")


Saved best model at epoch 1 with loss: 0.6207, accuracy: 68.27%
Epoch 1/300, Loss: 0.6207, Accuracy: 68.27%
Saved best model at epoch 3 with loss: 0.5883, accuracy: 69.64%
Saved best model at epoch 4 with loss: 0.5856, accuracy: 70.40%
Saved best model at epoch 6 with loss: 0.5727, accuracy: 67.93%
Saved best model at epoch 8 with loss: 0.5665, accuracy: 69.20%
Saved best model at epoch 10 with loss: 0.5594, accuracy: 67.45%
Saved best model at epoch 11 with loss: 0.5346, accuracy: 68.90%
Epoch 11/300, Loss: 0.5346, Accuracy: 68.90%
Saved best model at epoch 13 with loss: 0.5333, accuracy: 69.43%
Saved best model at epoch 14 with loss: 0.5251, accuracy: 68.71%
Saved best model at epoch 20 with loss: 0.4923, accuracy: 68.61%
Epoch 21/300, Loss: 0.5141, Accuracy: 68.35%
Saved best model at epoch 24 with loss: 0.4918, accuracy: 70.62%
Saved best model at epoch 25 with loss: 0.4892, accuracy: 70.34%
Saved best model at epoch 29 with loss: 0.4721, accuracy: 71.94%
Epoch 31/300, Loss: 0.4888

In [ ]:
model.eval()  # Đặt mô hình ở chế độ đánh giá (evaluation)

# Kiểm tra trên tập kiểm tra (test set)
with torch.no_grad():  # Không tính gradient khi đánh giá
    out_test = model(graph_test.x, graph_test.edge_index)  # Áp dụng mô hình lên tập test
    
    # Tính toán loss trên tập test
    loss_test = criterion(out_test, graph_test.y)  # Tính loss (CrossEntropyLoss)
    
    # Tính toán accuracy trên tập test
    _, predicted_test = torch.max(out_test, 1)  # Lấy chỉ số của lớp dự đoán có xác suất cao nhất
    correct_test = (predicted_test == graph_test.y).sum().item()  # Tính số lượng dự đoán đúng
    accuracy_test = correct_test / graph_test.y.size(0)  # Tính accuracy trên tập test

    print(f"Best Model Test Loss: {loss_test.item():.4f}")
    print(f"Best Model Test Accuracy: {accuracy_test * 100:.2f}%")

Best Model Test Loss: 0.5016
Best Model Test Accuracy: 70.68%


# Đánh giá

In [ ]:
def get_product_recommendations(user_id, matrix, model, graph_data, probability_threshold=0.6, num_suggestions=10):
    with torch.no_grad():
        predictions = model(graph_data.x, graph_data.edge_index)
        product_probs = torch.softmax(predictions[len(matrix):], dim=1)[:, 1].cpu().numpy()

    purchased_products = matrix.loc[user_id][matrix.loc[user_id] > 0].index.tolist()

    predicted_products = [
        col for col, prob in zip(matrix.columns, product_probs)
        if prob > probability_threshold and col not in purchased_products
    ]

    predicted_products_sorted = sorted(predicted_products, key=lambda x: product_probs[matrix.columns.get_loc(x)], reverse=True)

    recommended_products = predicted_products_sorted[:num_suggestions]

    return recommended_products


In [22]:
# Test hàm
user_id = 15463  # Giả sử bạn muốn gợi ý cho User 0
recommended_products = get_product_recommendations(user_id, test_matrix, model, graph_test)

print("Gợi ý sản phẩm cho User", user_id, ":", recommended_products)
len(recommended_products)

Gợi ý sản phẩm cho User 15463 : [(23565, 0.99515355), (23481, 0.99657357), (72128, 0.998004), (22102, 0.99658453), (20914, 0.99612033), (23527, 0.9939475), (9021418, 0.9969183), (22824, 0.9950186), (84754, 0.99550396), (21158, 0.9973979)]


10

# Recommend products customers

In [ ]:
import pandas as pd
from tqdm import tqdm

user_ids = data_matrix.index.tolist()

all_recommendations = []

for user_id in tqdm(user_ids, desc="Processing Users", unit="user"):
    recommended_products = get_product_recommendations(user_id, data_matrix, model, data, product_weights)
    all_recommendations.append({'user_id': user_id, 'recommended_products': recommended_products})

recommendations_df = pd.DataFrame(all_recommendations)

recommendations_df.to_csv('user_product_recommendations_gnn_no_util.csv', index=False)


Processing Users: 100%|██████████| 4312/4312 [45:58<00:00,  1.56user/s] 

Gợi ý sản phẩm cho các user đã được lưu vào file 'user_product_recommendations_gnn_util.csv'.


# Lợi nhuận trung bình tất cả user

In [ ]:
import pandas as pd
from tqdm import tqdm

recommendations_df = pd.read_csv('user_product_recommendations_gnn_no_util.csv')
items_df = pd.read_csv('items.csv')

items_df['stockCodeTransform'] = items_df['stockCodeTransform'].astype(str)

user_values = []

for user_id in tqdm(recommendations_df['user_id'], desc="Processing Users", unit="user"):
    recommended_products = recommendations_df[recommendations_df['user_id'] == user_id]['recommended_products'].values[0]
    
    if pd.isna(recommended_products):
        continue
    
    recommended_products = [str(product[0]) for product in eval(recommended_products)]
    
    total_value = 0
    for product in recommended_products:
        product = product.strip()
        product_info = items_df[items_df['stockCodeTransform'] == product]
        if not product_info.empty:
            unit_price = product_info['UnitPrice'].values[0]
            quantity = product_info['Quantity'].values[0]
            product_value = unit_price * quantity
            total_value += product_value
    
    if len(recommended_products) > 0:
        avg_value = total_value / len(recommended_products)
        user_values.append(avg_value)

average_value = sum(user_values) / len(user_values) if user_values else 0

print(f"Giá trị trung bình của tất cả user: {average_value:.2f}")


Processing Users: 100%|██████████| 4312/4312 [00:18<00:00, 235.07user/s]

Giá trị trung bình của tất cả user: 22.03
